In [1]:
from os import environ

In [11]:
import pickle
import os
import json

import cv2

from spatialyze.geospatial_video import GeospatialVideo
from spatialyze.road_network import RoadNetwork
from spatialyze.world import World
from spatialyze.video_processor.camera_config import camera_config
from spatialyze.utils.F import distance, contains, heading_diff, has_types, stopped, left_turn, view_angle
from utils import topdown_config, vv_config, resize

ModuleNotFoundError: No module named 'utils'

In [12]:
DATA_DIR = '../../../data'
OUTPUT_DIR = os.path.join(DATA_DIR, 'pipeline/outputs')
VIDEO_DIR = os.path.join(DATA_DIR, 'pipeline/videos')
ROAD_DIR = os.path.join(DATA_DIR, 'scenic/road-network/boston-seaport')

files = os.listdir(VIDEO_DIR)
with open(os.path.join(VIDEO_DIR, 'frames.pkl'), 'rb') as f:
    videos = pickle.load(f)

In [16]:
world = World()
world.addGeogConstructs(RoadNetwork('Boston-Seaport', ROAD_DIR))

for file in os.listdir(VIDEO_DIR):
    if not file.endswith('.camera.pkl'):
        continue

    with open(os.path.join(VIDEO_DIR, file), 'rb') as f:
        camera = pickle.load(f)
    videofile = os.path.join(VIDEO_DIR, camera['filename'])
    camera = [camera_config(*c) for c in camera['frames']]

    world.addVideo(GeospatialVideo(videofile, camera))

car = world.object()
pedestrian = world.object()
camera = world.camera()
intersection = world.geogConstruct(type='intersection')
road = world.geogConstruct(type='road')

world.filter(
    has_types(car, 'car') &
    (distance(camera, car) < 50) &
    (view_angle(camera, car) < 135) &
    has_types(pedestrian, 'pedestrian') &
    heading_diff(camera, pedestrian, between=[-180, 180]) & 
    (contains(intersection, [car, pedestrian]) |
        contains(road, [car, pedestrian])) &
    ~heading_diff(road, car, between=[-70, 70])
)



In [15]:
world.saveVideos(os.path.join(OUTPUT_DIR, 'figure12'), addBoundingBoxes=True)

AssertionError: CallNode(_fn=(<function road_segment at 0x107d01cf0>,), params=[LiteralNode(value='intersection', python=True)])